# CS- 2   S-DES

In [1]:

# Initial Permutation Table
IP = [2, 6, 3, 1, 4, 8, 5, 7]

# Inverse Initial Permutation Table
IP_INV = [4, 1, 3, 5, 7, 2, 8, 6]

# Expansion Permutation Table
EP = [4, 1, 2, 3, 2, 3, 4, 1]

# Permutation Function P4
P4 = [2, 4, 3, 1]

# Permutation Function P8
P8 = [6, 3, 7, 4, 8, 5, 10, 9]

# Permutation Function P10
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]

# S-Box Tables
S0 = [
    [1, 0, 3, 2],
    [3, 2, 1, 0],
    [0, 2, 1, 3],
    [3, 1, 3, 2]
]

S1 = [
    [0, 1, 2, 3],
    [2, 0, 1, 3],
    [3, 0, 1, 0],
    [2, 1, 0, 3]
]

def permute(input_bits, permutation_table):
    return ''.join(input_bits[i - 1] for i in permutation_table)

def generate_keys(key):
    key = permute(key, P10)
    left_key = key[:5]
    right_key = key[5:]

    # Left Circular Shifts
    left_key = left_key[1:] + left_key[0]
    right_key = right_key[1:] + right_key[0]

    key1 = permute(left_key + right_key, P8)

    # Double Left Circular Shifts
    left_key = left_key[2:] + left_key[:2]
    right_key = right_key[2:] + right_key[:2]

    key2 = permute(left_key + right_key, P8)

    return key1, key2

def xor(a, b):
    return ''.join('1' if x != y else '0' for x, y in zip(a, b))

def sbox(sub_block, s_box):
    row = int(sub_block[0] + sub_block[3], 2)
    col = int(sub_block[1:3], 2)
    return format(s_box[row][col], '02b')

def f_k(sub_block, key):
    expanded_block = permute(sub_block, EP)
    xored_block = xor(expanded_block, key)

    left_block = xored_block[:4]
    right_block = xored_block[4:]

    sbox_output = sbox(left_block, S0) + sbox(right_block, S1)

    return permute(sbox_output, P4)

def swap_bits(block):
    return block[4:] + block[:4]

def encrypt(input_text, key):
    print("Initial permutation:")
    input_text = permute(input_text, IP)
    print(input_text)

    key1, key2 = generate_keys(key)
    print("Key 1:", key1)
    print("Key 2:", key2)

    left_block = input_text[:4]
    right_block = input_text[4:]

    f_k1_output = f_k(right_block, key1)
    print("Fk1 output:", f_k1_output)
    xored_output = xor(left_block, f_k1_output)
    print("XOR with left block:", xored_output)

    left_block = right_block
    right_block = xored_output

    left_block, right_block = right_block, left_block

    f_k2_output = f_k(right_block, key2)
    print("Fk2 output:", f_k2_output)
    xored_output = xor(left_block, f_k2_output)
    print("XOR with left block:", xored_output)

    cipher_text = permute(xored_output + right_block, IP_INV)
    print("Cipher text:", cipher_text)

    return cipher_text

def decrypt(cipher_text, key):
    print("Initial permutation:")
    cipher_text = permute(cipher_text, IP)
    print(cipher_text)

    key1, key2 = generate_keys(key)
    print("Key 1:", key1)
    print("Key 2:", key2)

    left_block = cipher_text[:4]
    right_block = cipher_text[4:]

    f_k2_output = f_k(right_block, key2)
    print("Fk2 output:", f_k2_output)
    xored_output = xor(left_block, f_k2_output)
    print("XOR with left block:", xored_output)

    left_block = right_block
    right_block = xored_output

    left_block, right_block = right_block, left_block

    f_k1_output = f_k(right_block, key1)
    print("Fk1 output:", f_k1_output)
    xored_output = xor(left_block, f_k1_output)
    print("XOR with left block:", xored_output)

    plain_text = permute(xored_output + right_block, IP_INV)
    print("Plain text:", plain_text)

    return plain_text

# Example usage
input_text = '10100010'  # 8-bit input text
key = '1010000010'       # 10-bit key

print("Encrypting...")
# Encryption
encrypted_text = encrypt(input_text, key)

print("\nDecrypting...")
# Decryption
decrypted_text = decrypt(encrypted_text, key)


Encrypting...
Initial permutation:
00110001
Key 1: 10100100
Key 2: 01000011
Fk1 output: 0110
XOR with left block: 0101
Fk2 output: 1010
XOR with left block: 1111
Cipher text: 11100110

Decrypting...
Initial permutation:
11110001
Key 1: 10100100
Key 2: 01000011
Fk2 output: 1010
XOR with left block: 0101
Fk1 output: 0110
XOR with left block: 0011
Plain text: 10100010
